In [11]:
from dataclasses import dataclass, asdict
from typing import Optional, Dict
from pathlib import Path
import json
import numpy as np


@dataclass
class ParameterBound:
    name: str
    min_value: float
    max_value: float
    resolution: float
    unit: str = ""

    def __post_init__(self):
        if self.min_value >= self.max_value:
            raise ValueError(
                f"Parameter {self.name}: min_value ({self.min_value}) must be less than max_value ({self.max_value})"
            )


    def grid(self) -> np.ndarray:
        """
        min_value ~ max_value 사이를 resolution 간격으로 나눈 grid
        (max_value 포함 보장)
        """
        n = int(np.floor((self.max_value - self.min_value) / self.resolution))
        values = self.min_value + self.resolution * np.arange(n + 1)

        # 수치 오차로 max_value를 못 넘겼으면 추가
        if values[-1] < self.max_value - 1e-12:
            values = np.append(values, self.max_value)

        return values

    def sample(self) -> float:
        """
        resolution grid 중에서 하나를 랜덤 샘플링
        """
        values = self.grid()
        return float(np.random.choice(values))

    def snap(self, value: float) -> float:
        """
        아무 값이나 resolution grid에 맞게 스냅
        """
        idx = round((value - self.min_value) / self.resolution)
        snapped = self.min_value + idx * self.resolution
        return float(np.clip(snapped, self.min_value, self.max_value))



class BoundStore:
    """ParameterBound들을 JSON 파일로 저장/로드하는 저장소"""

    def __init__(self, path: str | Path):
        self.path = Path(path)
        self.data: Dict[str, dict] = self._load_all()

    def _load_all(self) -> Dict[str, dict]:
        if not self.path.exists():
            return {}
        return json.loads(self.path.read_text(encoding="utf-8"))

    def _save_all(self) -> None:
        self.path.parent.mkdir(parents=True, exist_ok=True)
        self.path.write_text(json.dumps(self.data, ensure_ascii=False, indent=2), encoding="utf-8")

    def get(self, default: ParameterBound) -> ParameterBound:
        saved = self.data.get(default.name)
        if not saved:
            self.data[default.name] = asdict(default)
            self._save_all()
            return default

        return ParameterBound(
            name=default.name,
            min_value=float(saved.get("min_value", default.min_value)),
            max_value=float(saved.get("max_value", default.max_value)),
            resolution=float(saved.get("resolution", default.resolution)),
            unit=saved.get("unit", default.unit),
        )

    def put(self, bound: ParameterBound) -> None:
        """현재 bound를 저장"""
        self.data[bound.name] = asdict(bound)
        self._save_all()


In [32]:
store = BoundStore("bounds.json")

# 1) 하드코딩된 기본 정의들
defaults = {
    "Tx_turns": ParameterBound(
        name="Tx_turns",
        min_value=3,
        max_value=10,
        resolution=1,
        unit="turn"
    ),
    "Rx_turns": ParameterBound(
        name="Rx_turns",
        min_value=3,
        max_value=10,
        resolution=1,
        unit="turn"
    ),
    "w1": ParameterBound(
        name="w1",
        min_value=100.0,
        max_value=300.0,
        resolution=0.1,
        unit="mm"
    ),
}

# 2) 저장된 값이 있으면 로드, 없으면 기본값 사용
bounds = {name: store.get(pb) for name, pb in defaults.items()}

# 확인
print(bounds["Tx_turns"])
print(bounds["Rx_turns"])


# Tx_turns 조정
bounds["Tx_turns"].min_value = 5
bounds["Tx_turns"].max_value = 8

# Rx_turns 조정
bounds["Rx_turns"].min_value = 4
bounds["Rx_turns"].max_value = 9


tx_val = bounds["Tx_turns"].sample()
rx_val = bounds["Rx_turns"].sample()

print(tx_val, rx_val)


w1 = bounds["w1"].snap(109.53)
print(w1)

ParameterBound(name='Tx_turns', min_value=3.0, max_value=10.0, resolution=1.0, unit='turn')
ParameterBound(name='Rx_turns', min_value=3.0, max_value=10.0, resolution=1.0, unit='turn')
5.0 4.0
109.5
